## Definir Claves

In [1]:
from dotenv import load_dotenv
import os

# Cargar .env
load_dotenv()

sea_key = os.getenv("SEA_KEY")
sub_key = os.getenv("SUB_KEY")
endpoint_name = os.getenv("ENDPOINT_NAME")
deployment_name = os.getenv("DEPLOYMENT_NAME")
search_endpoint_name = os.getenv("SEARCH_ENDPOINT_NAME")
blob_sas_url_name = os.getenv("BLOB_SAS_URL_NAME")

## Interfaz

In [2]:
import os
import re
import markdown2
from openai import AzureOpenAI
from IPython.display import HTML, display, clear_output
from ipywidgets import widgets

# Cargar variabls
endpoint = endpoint_name
deployment = deployment_name
search_endpoint = search_endpoint_name
search_key = sea_key
subscription_key = sub_key

# URL base del blob storage con la clave SAS
blob_sas_url = blob_sas_url_name

# Inicializar el cliente de Azure OpenAI
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=subscription_key,
    api_version="2025-01-01-preview",
)

# Leer papel del agente
with open("role.md", "r", encoding="utf-8") as file:
    role = file.read()

# Mensaje del sistema
system_message = role

# Ahora puedes usar 'contenido_md' como una variable con el texto del archivo
print()

def render_markdown(text):
    """Convierte texto markdown a HTML"""
    try:
        # Usar markdown2 para convertir a HTML
        return markdown2.markdown(text, extras=["tables", "fenced-code-blocks"])
    except Exception as e:
        # Si hay error, devolver el texto con formato básico
        print(f"Error al renderizar markdown: {e}")
        return text.replace('\n', '<br>')

def consultar_asistente(consulta, historial=None):
    if historial is None:
        historial = [{"role": "system", "content": system_message}]
    
    # Añadir la nueva consulta al historial
    historial.append({"role": "user", "content": consulta})
    
    # Generar la completación
    completion = client.chat.completions.create(
        model=deployment,
        messages=historial,
        max_tokens=2500,
        temperature=0.2,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        extra_body={
          "data_sources": [{
              "type": "azure_search",
              "parameters": {
                "endpoint": search_endpoint,
                "index_name": "knowledgellmchoferes",
                "authentication": {
                  "type": "api_key",
                  "key": search_key
                },
                "query_type": "simple",
                "in_scope": True,
                "top_n_documents": 20
              }
            }]
        }
    )
    
    # Añadir la respuesta al historial
    respuesta = completion.choices[0].message.content
    historial.append({"role": "assistant", "content": respuesta})
    
    return completion, historial

def estandarizar_referencias(content):
    """
    Reemplaza [docN] por [N] en el texto
    """
    # Patrón para encontrar referencias [docN]
    pattern = r'\[doc(\d+)\]'
    
    # Función para reemplazar cada coincidencia
    def replace_match(match):
        num = match.group(1)
        return f'[{num}]'
    
    # Realizar el reemplazo
    standardized_content = re.sub(pattern, replace_match, content)
    
    return standardized_content

def generar_url_documento(filepath):
    """
    Genera una URL para descargar un documento desde el blob storage
    """
    # Escapar caracteres especiales en el nombre del archivo
    import urllib.parse
    
    # Ajustar la ruta para incluir la carpeta "documentos/"
    filepath_completo = f"documentos/{filepath}"
    filename_encoded = urllib.parse.quote(filepath_completo)
    
    # Extraer la parte de la URL base y el token SAS
    container_url = blob_sas_url.split('?')[0]  # URL del contenedor sin el token
    sas_token = blob_sas_url.split('?')[1]      # Token SAS
    
    # Construir la URL completa
    document_url = f"{container_url}/{filename_encoded}?{sas_token}"
    
    return document_url

def generar_html_respuesta(completion):
    """
    Genera HTML para la respuesta del asistente con markdown y referencias.
    """
    # Extraer el contenido de la respuesta
    content = completion.choices[0].message.content
    
    # Estandarizar las referencias en el texto [docN] -> [N]
    content = estandarizar_referencias(content)
    
    # Convertir markdown a HTML
    html_content = render_markdown(content)
    
    # Extraer las citas/fuentes utilizadas
    referencias = []
    
    # Obtener la estructura completa como diccionario
    completion_dict = completion.model_dump()
    
    # Extraer referencias de la estructura de diccionario
    if (completion_dict.get('choices') and len(completion_dict['choices']) > 0 and
        completion_dict['choices'][0].get('message') and
        completion_dict['choices'][0]['message'].get('context') and
        completion_dict['choices'][0]['message']['context'].get('citations')):
        
        citations = completion_dict['choices'][0]['message']['context']['citations']
        
        for citation in citations:
            if citation.get('filepath'):
                referencias.append({
                    'filepath': citation.get('filepath', ''),
                    'chunk_id': citation.get('chunk_id', ''),
                    'title': citation.get('title', '')
                })
    
    # Construir HTML para la respuesta
    html_output = "<div style='font-family: Arial, sans-serif; width: 100%; max-width: 780px; margin: 0 auto; word-wrap: break-word; overflow-wrap: break-word; margin-bottom: 20px;'>"
    
    # Título y respuesta principal
    html_output += "<div style='background-color: #f0f7ff; padding: 20px; border-radius: 10px; margin-bottom: 20px;'>"
    html_output += "<h2 style='color: #0066cc; margin-top: 0;'>🤖 Respuesta del Asistente</h2>"
    html_output += f"<div style='overflow-x: hidden;'>{html_content}</div>"
    html_output += "</div>"
    
    # Información de documentos consultados
    if referencias:
        html_output += "<div style='background-color: #f5f5f5; padding: 20px; border-radius: 10px;'>"
        html_output += "<h3 style='color: #444; margin-top: 0;'>📚 Documentos Consultados</h3>"
        html_output += "<ol style='padding-left: 20px;'>"  # Cambiado a lista ordenada (ol)
        
        # Filtrar referencias duplicadas por filepath
        filepath_set = set()
        
        for ref in referencias:
            filepath = ref['filepath']
            if filepath not in filepath_set:
                filepath_set.add(filepath)
                # Generar URL para el documento
                doc_url = generar_url_documento(filepath)
                # Añadir hipervínculo con estilo para evitar desbordamiento
                html_output += f"<li style='word-break: break-word;'><strong><a href='{doc_url}' target='_blank'>{filepath}</a></strong>"
        
        html_output += "</ol>"
        html_output += "</div>"
    else:
        html_output += "<div style='background-color: #f5f5f5; padding: 20px; border-radius: 10px;'>"
        html_output += "<h3 style='color: #444; margin-top: 0;'>📚 Documentos Consultados</h3>"
        html_output += "<p>No se encontraron referencias a documentos específicos.</p>"
        html_output += "</div>"
    
    # Estadísticas
    if hasattr(completion, 'usage'):
        html_output += "<div style='margin-top: 20px; color: #777; font-size: 0.9em;'>"
        html_output += f"<p>Tokens totales: {completion.usage.total_tokens} | "
        html_output += f"Prompt: {completion.usage.prompt_tokens} | "
        html_output += f"Respuesta: {completion.usage.completion_tokens}</p>"
        html_output += "</div>"
    
    html_output += "</div>"
    
    return html_output

def interfaz_interactiva_con_historial():
    """
    Interfaz interactiva con historial de conversación desplazable
    """
    # Definir el CSS para la interfaz
    css = """
    <style>
        /* Estilos para la interfaz principal */
        .app-container {
            max-width: 800px;
            margin: 0 auto;
            font-family: Arial, sans-serif;
            overflow-x: hidden;
        }
        
        /* Contenedor de conversación con scroll vertical */
        .conversation-scroll {
            max-height: 600px;
            overflow-y: auto;
            overflow-x: hidden;
            border: 1px solid #eaeaea;
            border-radius: 10px;
            padding: 10px;
            margin-bottom: 20px;
            background-color: #ffffff;
            width: 100%;
            max-width: 780px;
        }
        
        /* Estilos para mensajes y links dentro del contenedor */
        .conversation-scroll a {
            word-break: break-all;
        }
        
        .conversation-scroll img, 
        .conversation-scroll table,
        .conversation-scroll pre {
            max-width: 100%;
            overflow-x: auto;
        }
        
        /* Animación para el indicador de carga */
        @keyframes spin {
            0% { transform: rotate(0deg); }
            100% { transform: rotate(360deg); }
        }
        
        .loading-spinner {
            border: 5px solid #f3f3f3;
            border-top: 5px solid #3498db;
            border-radius: 50%;
            width: 30px;
            height: 30px;
            animation: spin 2s linear infinite;
        }
    </style>
    """
    display(HTML(css))
    
    # Crear el contenedor principal con ancho controlado
    container = widgets.VBox([], layout=widgets.Layout(max_width='1024px', margin='0 auto'))
    
    # Crear el encabezado
    header = widgets.HTML("""
    <div class="app-container">
        <div style="background-color: #0066cc; color: white; padding: 15px; border-radius: 10px; margin-bottom: 20px; text-align: center;">
            <h1 style="margin: 0;">🚚 Asistente para Choferes</h1>
            <p style="margin: 5px 0 0 0;">Pregúntame sobre cualquier tema relacionado con normativas, detenciones o documentación</p>
        </div>
    </div>
    """)
    
    # Crear el contenedor para el historial de conversación con estilo scrollable
    scrollable_container = widgets.HTML('<div class="app-container"><div class="conversation-scroll" id="conversation-container"></div></div>')
    
    # Widget de entrada de texto
    consulta_input = widgets.Textarea(
        value='',
        placeholder='Escribe tu pregunta aquí...',
        description='',
        disabled=False,
        layout=widgets.Layout(width='100%', height='100px', max_width='800px')
    )
    
    # Botón de enviar
    enviar_btn = widgets.Button(
        description='Enviar Consulta',
        button_style='primary',
        tooltip='Haz clic para enviar tu consulta',
        icon='paper-plane'
    )
    
    # Botón para limpiar el historial
    clear_btn = widgets.Button(
        description='Limpiar Historial',
        button_style='danger',
        tooltip='Haz clic para limpiar el historial de conversación',
        icon='trash'
    )
    
    # Contenedor para mensajes
    messages_output = widgets.Output()
    
    # Historial de mensajes de la API OpenAI
    api_historial = None
    # Contador para IDs únicos
    message_counter = 0
    
    # Función para añadir mensaje al contenedor
    def add_message_to_container(html_content):
        nonlocal message_counter
        message_id = f"message-{message_counter}"
        message_counter += 1
        
        with messages_output:
            display(HTML(f"""
            <script>
                var container = document.getElementById('conversation-container');
                if (container) {{
                    var newMessage = document.createElement('div');
                    newMessage.id = "{message_id}";
                    newMessage.innerHTML = `{html_content}`;
                    container.appendChild(newMessage);
                    container.scrollTop = container.scrollHeight;
                }}
            </script>
            """))
        
        return message_id
    
    # Función para actualizar un mensaje existente
    def update_message(message_id, html_content):
        with messages_output:
            display(HTML(f"""
            <script>
                var message = document.getElementById('{message_id}');
                if (message) {{
                    message.innerHTML = `{html_content}`;
                    var container = document.getElementById('conversation-container');
                    if (container) {{
                        container.scrollTop = container.scrollHeight;
                    }}
                }}
            </script>
            """))
    
    # Inicializar con un mensaje de bienvenida
    add_message_to_container("""
    <div style="text-align: center; padding: 20px; color: #666; word-wrap: break-word;">
        <p>¡Bienvenido al Asistente para Choferes! Escribe tu consulta y haz clic en 'Enviar Consulta' para comenzar.</p>
    </div>
    """)
    
    # Función para manejar el clic en el botón enviar
    def on_enviar_click(b):
        consulta = consulta_input.value
        if consulta.strip():
            nonlocal api_historial
            
            # Escapar caracteres especiales para JavaScript
            consulta_escapada = consulta.replace('\\', '\\\\').replace('`', '\\`').replace('$', '\\$')
            
            # Mostrar la pregunta del usuario
            question_html = f"""
            <div style="margin: 10px 0; padding: 10px; background-color: #e6f2ff; border-radius: 10px; word-wrap: break-word; overflow-wrap: break-word;">
                <strong>🙋 Tu pregunta:</strong>
                <p style="margin: 5px 0 0 0;">{consulta_escapada}</p>
            </div>
            """
            add_message_to_container(question_html)
            
            # Mostrar indicador de carga
            loading_id = add_message_to_container("""
            <div style="display: flex; justify-content: center; margin: 20px 0;">
                <div class="loading-spinner"></div>
            </div>
            """)
            
            # Procesar la consulta
            try:
                completion, api_historial = consultar_asistente(consulta, api_historial)
                
                # Generar HTML para la respuesta
                response_html = generar_html_respuesta(completion)
                
                # Reemplazar el indicador de carga con la respuesta
                # Escapar caracteres especiales para JavaScript
                response_html_escapado = response_html.replace('\\', '\\\\').replace('`', '\\`').replace('$', '\\$')
                update_message(loading_id, response_html_escapado)
                
            except Exception as e:
                # Mostrar mensaje de error en lugar del indicador de carga
                error_html = f"""
                <div style="margin: 10px 0; padding: 10px; background-color: #ffe6e6; border-radius: 10px; word-wrap: break-word;">
                    <strong>❌ Error:</strong>
                    <p style="margin: 5px 0 0 0;">Ocurrió un error al procesar tu consulta: {str(e)}</p>
                </div>
                """
                update_message(loading_id, error_html)
            
            # Limpiar el campo de entrada
            consulta_input.value = ''
    
    # Función para manejar el clic en el botón limpiar
    def on_clear_click(b):
        nonlocal api_historial, message_counter
        
        # Reiniciar el historial de la API
        api_historial = None
        message_counter = 0
        
        # Limpiar el contenedor de conversación
        with messages_output:
            display(HTML("""
            <script>
                var container = document.getElementById('conversation-container');
                if (container) {
                    container.innerHTML = '';
                }
            </script>
            """))
        
        # Mostrar mensaje de que se limpió el historial
        add_message_to_container("""
        <div style="text-align: center; padding: 20px; color: #666;">
            <p>Historial de conversación limpiado.</p>
        </div>
        """)
    
    # Vincular funciones a los botones
    enviar_btn.on_click(on_enviar_click)
    clear_btn.on_click(on_clear_click)
    
    # Crear layout para los botones
    buttons_container = widgets.HTML("""
    <div class="app-container" style="display: flex; justify-content: space-between;">
        <div id="buttons-container"></div>
    </div>
    """)
    
    btn_layout = widgets.HBox([enviar_btn, clear_btn], layout=widgets.Layout(width='100%', justify_content='space-between', max_width='800px'))
    
    # Añadir widgets al contenedor principal
    container.children = [
        header,
        scrollable_container,
        messages_output,
        widgets.VBox([consulta_input], layout=widgets.Layout(display='flex', justify_content='center', align_items='center')),
        widgets.VBox([btn_layout], layout=widgets.Layout(display='flex', justify_content='center', align_items='center'))
    ]
    
    # Mostrar la interfaz
    display(container)

# Ejecutar la interfaz interactiva
interfaz_interactiva_con_historial()